In [ ]:
import pandas  as pd
import matplotlib.pyplot as plt
import numpy as np


from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from health.utils import load_training_data, column_encoder
from sklearn.neural_network import MLPClassifier
from joblib import dump


df = load_training_data('v1')
df = df.reset_index(drop=True)
df.set_index(['id'])


col_len = len(df.columns.to_list())
X = df.iloc[:,1:col_len -1]
Y = df.iloc[:,col_len-1:]



In [ ]:

# Transform the data
X_encoded = column_encoder.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.25, random_state=16)

# s1= set(X_k_test['Profession'].unique())
# s2 = set(X['Profession'].unique())
# print('s1-s2', s1.difference(s2))
# print('s2-s1' , s2.difference(s1))

# instantiate the model (using the default parameters)
model = LogisticRegression(random_state=16)
# # fit the model with data

model.fit(X_train, y_train)
# print(X_encoded)
# print(X_test)  
dump(model, '../models/logistic-features-not-combined-rs-16.joblib')
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)

X_encoded = column_encoder.fit_transform(X)
print(report)


In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 3), random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.25, random_state=16)
clf.fit(X_train, y_train)
clf.predict(X_test)
dump(clf,'../models/mlp-clf-01.joblib')
report = classification_report(y_test, y_pred)
print(report)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
 
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, random_state=42)
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)  # apply scaling on training data
pipe.score(X_test, y_test)  # apply scaling on testing data, without leaking training data.